In [1]:
import common.ipynb_importer
from db.pg.pg_00_common import *

cursor = pg_connect()

importing Jupyter notebook from D:\sourcecode\keep_learning\db\pg\pg_00_common.ipynb


# Function return a table

语法
```
create or replace function function_name (
   parameter_list
) 
returns table ( column_list ) 
language plpgsql
as 
$$
declare 
-- variable declaration
begin
-- body
end; 
$$;
```

In [2]:
sql = """
create or replace function get_film (
  p_pattern varchar
) 
returns table (
	film_title varchar,
	film_release_year int
) 
language plpgsql
as $$
begin
	return query 
		select
			title,
			release_year::integer
		from
			film
		where
			title ilike p_pattern;
end;
$$;
"""
cursor.execute(sql)

<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=localhost user=postgres database=dvdrental) at 0x2101e49b110>

In [3]:
sql = """
SELECT * FROM get_film ('Al%');
"""
run_sql(cursor, sql)

         film_title  film_release_year
0     Alabama Devil               2006
1  Aladdin Calendar               2006
2   Alamo Videotape               2006
3    Alaska Phantom               2006
4       Ali Forever               2006
5    Alice Fantasia               2006
6      Alien Center               2006
7   Alley Evolution               2006
8        Alone Trip               2006
9     Alter Victory               2006


In [4]:
sql = """
create or replace function get_film (
	p_pattern varchar,
	p_year int
) 
returns table (
	film_title varchar,
	film_release_year int
) 
language plpgsql
as $$
declare 
    var_r record;
begin
	for var_r in(
            select title, release_year 
            from film 
	        where title ilike p_pattern and release_year = p_year
        ) loop 
            film_title := upper(var_r.title) ; 
            film_release_year := var_r.release_year;
            return next;
	end loop;
end; $$ 
"""
cursor.execute(sql)

<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=localhost user=postgres database=dvdrental) at 0x2101e49b110>

In [5]:
sql = """
SELECT * FROM get_film ('%er', 2006);
"""
run_sql(cursor, sql)

                     film_title  film_release_year
0                ACE GOLDFINGER               2006
1                   ALI FOREVER               2006
2                  ALIEN CENTER               2006
3             AMISTAD MIDSUMMER               2006
4   ARACHNOPHOBIA ROLLERCOASTER               2006
..                          ...                ...
73                WANDA CHAMBER               2006
74           WATERSHIP FRONTIER               2006
75                WISDOM WORKER               2006
76                 WORDS HUNTER               2006
77                 WORST BANGER               2006

[78 rows x 2 columns]


# Function return a set of rows

语法：
```
create or replace function function_name(parameters)
returns setof row_structure
as
$$
   -- logic
   -- ...
   -- return one or more rows
   return query select_query;
$$ language plpgsql;
```

调用函数：
```
SELECT function_name(argument);
```

In [6]:
sql = """
create or replace function find_film_by_id(
	p_id int
)
returns setof film
as
$$
begin
   return query select * from film
   where film_id = p_id;
end;
$$
language plpgsql;
"""
cursor.execute(sql)

<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=localhost user=postgres database=dvdrental) at 0x2101e49b110>

In [7]:
sql = """
SELECT find_film_by_id(100);
"""
run_sql(cursor, sql)

                                     find_film_by_id
0  (100,"Brooklyn Desert","A Beautiful Drama of a...


In [8]:
sql = """
select (find_film_by_id(100)).title;
"""
run_sql(cursor, sql)

             title
0  Brooklyn Desert


In [9]:
sql = """
SELECT * FROM find_film_by_id(100);
"""
run_sql(cursor, sql)

   film_id            title  \
0      100  Brooklyn Desert   

                                         description  release_year  \
0  A Beautiful Drama of a Dentist And a Composer ...          2006   

   language_id  rental_duration rental_rate  length replacement_cost rating  \
0            1                7        4.99     161            21.99      R   

              last_update special_features  \
0 2013-05-26 14:50:58.951   [Commentaries]   

                                            fulltext  
0  'battl':14 'beauti':4 'brooklyn':1 'compos':11...  
